Source: https://github.com/oschow/take-a-hike/blob/master/AllTrails/scrape_clean/scrape_ratings.py

In [1]:
# installs
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpa

In [2]:
# imports
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, SoupStrainer
from pymongo import MongoClient
import time
import sys
import re
import urllib
import csv

In [3]:
# Set up chromedriver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [4]:
# Definition for logging in
def login(browser):
    browser.get('https://www.alltrails.com/login?ref=header')
    username = browser.find_element_by_name("userEmail")
    password = browser.find_element_by_name("userPassword")
    username.send_keys("221ornithomimus@gmail.com")
    password.send_keys("Dj3!120423u_9jkd_5!k34")
    browser.find_element_by_xpath("//input[@data-test-id='formButton-submit']").click()


# Scroll through the page and load all of the hikes
def get_hikes(browser, difficulty):
    if (difficulty != 'none'):
        browser.get('https://www.alltrails.com/canada/british-columbia')
        time.sleep(2)
        elements = browser.find_elements_by_xpath("//img[@alt='Caret']") # find the difficulty list
        time.sleep(2)
        elements[0].click() #click the difficulty list
        time.sleep(2)
        if (difficulty == 'easy'):
            difficulty_box = browser.find_element_by_xpath("//input[@value='1,2']") # find the difficulty checkboxes
            difficulty_box.click()
            if (difficulty_box.is_selected()): print("Selected: Easy")
            else: print("Error selecting difficulty")
        elif (difficulty == 'moderate'):
            difficulty_box = browser.find_element_by_xpath("//input[@value='3,4']")
            difficulty_box.click()
            if (difficulty_box.is_selected()): print("Selected: Moderate")
            else: print("Error selecting difficulty")
        elif (difficulty == 'hard'):
            difficulty_box = browser.find_element_by_xpath("//input[@value='5,6,7,8']")
            difficulty_box.click()
            if (difficulty_box.is_selected()): print("Selected: Hard")
            else: print("Error selecting difficulty")

    time.sleep(20)
    i = 0
    while True:
        try:
            element = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR,"[title^= 'Show more trails']" )))
            element.click()
            time.sleep(7)
            print("Getting hikes:", difficulty, str(i))
            i += 1
        except:
            break
    soup = BeautifulSoup(browser.page_source)
    return soup

# def get_photos(hike, browser):
#     browser.get(hike[0])
#     soup2 = BeautifulSoup(browser.page_source)
#     hikes = soup2.find_all(class_="photo-item gallery sm")[:3] # get first 3 photos
#     print(hikes)
#     i = 0
#     for element in hikes:
#         hike_url = element.get('href') # get url from hike
#         urllib.request.urlretrieve(hike_url, filename="/content/hike_photos/" + hike[1] + "-" + str(i) + ".jpg")
#         i += 1
#         time.sleep(5) # wait 5 seconds to avoid being blocked

# Get links to a bunch of photos
def get_photo_links(hike, browser):
    browser.get(hike[0])
    soup2 = BeautifulSoup(browser.page_source)
    hikes = soup2.find_all(class_="trail-photo lazyloaded")[:3] # get first 3 photos
    i = 0
    links = []
    for element in hikes:
        hike_url = element.get('src') # get url from hike
        links += [(hike_url, hike[1] + "-" + str(i))]
        i += 1
        time.sleep(1) # wait 1 second to avoid being blocked
    return links

In [5]:
# Log in to AllTrails
login(wd)

In [ ]:
# Scroll through and load all available easy hikes
easy_soup = get_hikes(wd, 'easy')
easy_hikes = easy_soup.find_all(itemid=re.compile('/trail/\S*'))

In [ ]:
# Get the hike urls for easy hikes
easy_hike_links = []

for hike in easy_hikes:
    tag = hike.get('itemid').split('/')[-1]
    easy_hike_links += [['https://www.alltrails.com' + hike.get('itemid') + '/photos', tag]]

# Save hike links to a csv file for backup
with open('easy_hike_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(easy_hike_links)

In [ ]:
# Scroll through all moderate hikes
moderate_soup = get_hikes(wd, 'moderate')
moderate_hikes = moderate_soup.find_all(itemid=re.compile('/trail/\S*'))

In [ ]:
# Get the hike urls for moderate hikes
moderate_hike_links = []

for hike in moderate_hikes:
    tag = hike.get('itemid').split('/')[-1]
    moderate_hike_links += [['https://www.alltrails.com' + hike.get('itemid') + '/photos', tag]]

# Save hike links to a csv file for backup
with open('moderate_hike_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(moderate_hike_links)

In [ ]:
# Get all photo links for easy hikes and save them to a csv file
easy_photo_links = []
for link in easy_hike_links:
    easy_photo_links += get_photo_links(link, wd)

# Save photo links to a csv file
with open('easy_photo_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(easy_photo_links)

In [ ]:
# Get all photo links for moderate hikes and save them to a csv file
moderate_photo_links = []
for link in moderate_hike_links:
    moderate_photo_links += get_photo_links(link, wd)

# Save photo links to a csv file
with open('moderate_photo_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(moderate_photo_links)

In [6]:
# Scroll through all hard hikes

hard_soup = get_hikes(wd, 'hard')
hard_hikes = hard_soup.find_all(itemid=re.compile('/trail/\S*'))

Selected: Hard
Getting hikes: hard 0
Getting hikes: hard 1
Getting hikes: hard 2
Getting hikes: hard 3
Getting hikes: hard 4
Getting hikes: hard 5
Getting hikes: hard 6
Getting hikes: hard 7
Getting hikes: hard 8
Getting hikes: hard 9
Getting hikes: hard 10
Getting hikes: hard 11
Getting hikes: hard 12
Getting hikes: hard 13
Getting hikes: hard 14
Getting hikes: hard 15
Getting hikes: hard 16
Getting hikes: hard 17
Getting hikes: hard 18
Getting hikes: hard 19
Getting hikes: hard 20
Getting hikes: hard 21
Getting hikes: hard 22
Getting hikes: hard 23
Getting hikes: hard 24
Getting hikes: hard 25
Getting hikes: hard 26
Getting hikes: hard 27
Getting hikes: hard 28
Getting hikes: hard 29
Getting hikes: hard 30
Getting hikes: hard 31
Getting hikes: hard 32
Getting hikes: hard 33
Getting hikes: hard 34
Getting hikes: hard 35
Getting hikes: hard 36
Getting hikes: hard 37
Getting hikes: hard 38
Getting hikes: hard 39
Getting hikes: hard 40
Getting hikes: hard 41
Getting hikes: hard 42
Gettin

In [7]:
# Get a list of hike urls and their id numbers
hard_hike_links = []

for hike in hard_hikes:
    tag = hike.get('itemid').split('/')[-1]
    hard_hike_links += [['https://www.alltrails.com' + hike.get('itemid') + '/photos', tag]]

# Save hike links to a csv file for backup
with open('hard_hike_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(hard_hike_links)

In [8]:
# Get all photo links for hard hikes and save them to a csv file
hard_photo_links = []
for link in hard_hike_links:
    hard_photo_links += get_photo_links(link, wd)

# Save photo links to a csv file
with open('hard_photo_links_3-29.csv', mode='w') as f:
    writer = csv.writer(f)
    writer.writerows(hard_photo_links)

VVV _____ Here be dragons _____ VVV

In [ ]:
wd.get('https://www.alltrails.com/canada/british-columbia')
time.sleep(2)
difficulties = wd.find_elements_by_xpath("//input[@type='checkbox']")
print(len(difficulties))
elements = wd.find_elements_by_xpath("//img[@alt='Caret']") # find the difficulty list
time.sleep(2)
elements[0].click() #click the difficulty list
time.sleep(5)
# element = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR,"[title^= 'Show more trails']" )))
difficulty = wd.find_element_by_xpath("//input[@value='5,6,7,8']") # find the difficulty checkboxes
print(difficulty.is_selected())
difficulty.click() # Click the checkbox matching the difficulty
print(difficulty.is_selected())
time.sleep(20)
element = WebDriverWait(wd, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR,"[title^= 'Show more trails']" )))
element.click()
time.sleep(7)

soup = BeautifulSoup(wd.page_source)
debug_hikes = soup.find_all(itemid=re.compile('/trail/\S*'))
tag = debug_hikes[10].get('itemid').split('/')[-1]
print(tag)

print(debug_hikes)

In [ ]:
# Select difficulty
wd.get('https://www.alltrails.com/canada/british-columbia')
time.sleep(1)
elements = wd.find_elements_by_xpath("//img[@alt='Caret']")
# browser.find_element_by_xpath("//input[@data-test-id='formButton-submit']").click()
# driver.find_element_by_xpath("//input[starts-with (@name,'Tut')]")
# xpath=//div[matches(@id,'che.*boxes')]
time.sleep(2)
#print(elements[0])
elements[0].click()
time.sleep(2)
easyButton = wd.find_elements_by_xpath("//input[@type='checkbox']")

In [ ]:
easyButton[0].click()

In [ ]:
# Download each image in the list
for hike in hike_links:
    get_photos(hike,wd)

In [ ]:
wd.get(hike_links[0][0])
soup2 = BeautifulSoup(wd.page_source)
# hikes = soup2.find_all(class='photo-item gallery sm')

In [ ]:
photos = soup2.find_all(class_="photo-item gallery sm")[:5]

In [ ]:
photos

In [ ]:
# Go to page
wd.get("https://www.alltrails.com/canada/british-columbia")

# For each entry, get itemid
soup = BeautifulSoup(wd.page_source)

hikes = soup.select('div data-hydrate')

In [ ]:
hikes = soup.select('div data-hydrate')

In [ ]:
hikes

[]